In [1]:
!pip install google-adk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.9/319.9 kB 10.6 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.0
    Uninstalling protobuf-6.33.0:
      Successfully uninstalled protobuf-6.33.0
  Attempting uninstall: cachetools
    Found existing installation: cachetools 6.2.1
    Uninstalling cachetools-6.2.1:
      Successfully uninstalled cachetools-6.2.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
google-cloud-translate 3.12.1 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.19.5, but you have protobuf 5.29.5 which is incompatible.
ray 2.51.1 requires click!=8.3.0,>=7.0, but you have click 8.3.0 which is incompatible.
bigframes 2.12.0 requires rich<14,>=12

In [747]:
import os
import json
from typing import Any, Dict, List
from google import genai
from google.adk.agents import LlmAgent
from google.genai import types
from google.adk.models.google_llm import Gemini
from google.adk.tools import AgentTool
from google.adk.tools.openapi_tool.openapi_spec_parser.openapi_toolset import OpenAPIToolset
from kaggle_secrets import UserSecretsClient
from google.adk.tools.openapi_tool.auth.auth_helpers import token_to_scheme_credential
from google.adk.runners import InMemoryRunner
from google.adk.sessions import InMemorySessionService
from google.adk.runners import Runner
from google.adk.tools.tool_context import ToolContext

import logging
# logging.getLogger("google_genai.types").setLevel(logging.ERROR)

In [650]:
GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY

retry_config = types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504],  # Retry on these HTTP errors
)

#### Defining the read_spec of 4 GRAPH Api : notebook, sections, pages and contents.
    - Graph API needs Bearer Tokens. Added Secrets to the notbook.

In [ ]:
onenote_read_spec = """
openapi: 3.0.0
info:
  title: Microsoft Graph OneNote Reader
  version: 1.0.0
servers:
  - url: https://graph.microsoft.com/v1.0
paths:
  # Level 1: Get Notebooks
  /me/onenote/notebooks:
    get:
      summary: List all notebooks
      operationId: list_notebooks
      responses:
        '200':
          description: List of notebooks
          content:
            application/json:
              schema:
                type: object
                properties:
                  value:
                    type: array
                    items:
                      type: object
                      properties:
                        id: {type: string}
                        displayName: {type: string}

  # Level 2: Get Sections within a Notebook
  /me/onenote/notebooks/{notebook_id}/sections:
    get:
      summary: List sections in a specific notebook
      operationId: list_sections
      parameters:
        - name: notebook_id
          in: path
          required: true
          schema: {type: string}
      responses:
        '200':
          description: List of sections
          content:
            application/json:
              schema:
                type: object
                properties:
                  value:
                    type: array
                    items:
                      type: object
                      properties:
                        id: {type: string}
                        displayName: {type: string}

  # Level 3: Get Pages within a Section
  /me/onenote/sections/{section_id}/pages:
    get:
      summary: List pages in a specific section
      operationId: list_pages
      parameters:
        - name: section_id
          in: path
          required: true
          schema: {type: string}
      responses:
        '200':
          description: List of pages
          content:
            application/json:
              schema:
                type: object
                properties:
                  value:
                    type: array
                    items:
                      type: object
                      properties:
                        id: {type: string}
                        title: {type: string}

  # Level 4: Get Actual HTML Content of a Page
  /me/onenote/pages/{page_id}/content:
    get:
      summary: Get the HTML content of a page
      operationId: get_page_content
      parameters:
        - name: page_id
          in: path
          required: true
          schema: {type: string}
      responses:
        '200':
          description: Page HTML content
          content:
            text/html:
              schema: {type: string}
"""

token = UserSecretsClient().get_secret("GRAPH_API_TOKEN")
# api_headers = {
#     "Authorization": f"Bearer {token}",
#     "Content-Type": "application/json"
# }

auth_scheme, auth_credential = token_to_scheme_credential(
        "apikey", 
        "header", 
        "Authorization", 
        f"Bearer {token}"
    )

#### Helper functions to see the Agents output and session state objects 

In [641]:
async def run_session(
    runner_instance: Runner,
    user_queries: list[str] | str = None,
    session_name: str = "default",
):
    print(f"\n ### Session: {session_name}")

    # Get app name from the Runner
    app_name = runner_instance.app_name

    # Attempt to create a new session or retrieve an existing one
    try:
        session = await session_service.create_session(
            app_name=app_name, user_id=USER_ID, session_id=session_name
        )
    except:
        session = await session_service.get_session(
            app_name=app_name, user_id=USER_ID, session_id=session_name
        )

    # Process queries if provided
    if user_queries:
        # Convert single query to list for uniform processing
        if type(user_queries) == str:
            user_queries = [user_queries]

        # Process each query in the list sequentially
        for query in user_queries:
            print(f"\nUser > {query}")

            # Convert the query string to the ADK Content format
            query = types.Content(role="user", parts=[types.Part(text=query)])

            # Stream the agent's response asynchronously
            async for event in runner_instance.run_async(
                user_id=USER_ID, session_id=session.id, new_message=query
            ):
                # Check if the event contains valid content
                if event.content and event.content.parts:
                    # Filter out empty or "None" responses before printing
                    # print(event.content.parts[0].text)
                    if (
                        event.content.parts[0].text != "None"
                        and event.content.parts[0].text
                    ):
                        print(f"Agent > {event.content.parts[0].text}")
    else:
        print("No queries!")


async def showSessionState(session_id="stateful-agentic-session"):
    session = await session_service.get_session(
        app_name=APP_NAME, user_id=USER_ID, session_id=session_id
    )
    
    print("Session State Contents:")
    print(json.dumps(session.state, indent=4))

In [ ]:
client = genai.Client(api_key=os.environ["GOOGLE_API_KEY"])

# print(f"{'Model ID':<30} | {'Display Name'}")
# print("-" * 60)

# for model in client.models.list():
#     # Filter for Gemini models to keep the list clean
#     if "gemini" in model.name:
#         # The API returns 'models/gemini-pro', we usually just need the ID part
#         model_id = model.name.split("/")[-1] 
#         print(f"{model_id:<30} | {model.display_name}")

MODEL_NAME = "gemini-2.5-flash-lite"

### Defining the Data Retrieval Agent
    - Uses the Cutom OpenApi Tool defined from the open_spec.?
    - Additional security on fetching notebooks. I have lot of notebooks I don't want to show here :)
    - OutPut format with Notebook Id as header works best for the other Agents to apply logic and reason on the user request.

In [773]:
toolset = OpenAPIToolset(
    spec_str=onenote_read_spec,
    spec_str_type='yaml',
    auth_scheme=auth_scheme,
    auth_credential=auth_credential
)


oneNote_api_agent = LlmAgent(
    name="oneNoteAPIAgent",
    model=Gemini(model=MODEL_NAME, retry_config=retry_config),
    tools=[toolset],
    instruction="""
        You are a OneNote API Agent. You have no write permissions.

        Based on the users request, you must fetch following content:
        - If the user request is about getting a notebook: 
            Call 'list_notebooks' to return notebook ids and its name.
        - If the user request is about getting the sections in a notebook: 
            Call 'list_sections' to return all the section names and its id using Notebook Id user provided.
        - If the user request is about getting the pages:
            Call 'list_pages' to return all the page ids and its name using the Section id user provides.
        - If the user request is about getting the page content:
            Call 'get_page_content' to return the text using the page id user gives.

        Restrictions: 
        Only do operations for Notebook name: "Google ADK". If the users asks for other notebooks, answer only "Google ADK" is available.
        
        CRITICAL OUTPUT FORMAT:
         When you successfully find the Notebook details, strictly format your final response like this:
        "Notebook Found: [Notebook Name] (ID: [Notebook ID]). Here are the details..."
        """
)

### Helper function tools for the Agents to save and retrieve the session data

In [774]:
## Notebook operations

def save_notebookinfo(
    tool_context: ToolContext, notebook_id: str, notebook_name: str
) -> Dict[str, Any]:
    """
    Tool to record and save notebook name and its Id.

    Args:
        notebook_id: The Id of the notebook to store in notebook session state
        notebook_name: The name of the notebook
    """
   
    tool_context.state["notebook:Id"] = notebook_id
    tool_context.state["notebook:Name"] = notebook_name

    return {"status": "success"}


def retrieve_notebookinfo(tool_context: ToolContext) -> Dict[str, Any]:
    """
    Tool to retrieve notebook name and its Id from session state.
    """

    notebook_id = tool_context.state.get("notebook:Id", None)
    notebook_name = tool_context.state.get("notebook:Name", None)

    
    return {"status": "success", "notebook_name": notebook_name, "notebook_id": notebook_id}


## Sections operations

def save_sectionsinfo(
    tool_context: ToolContext, notebook_id: str, section_ids: List[str], section_names: List[str]
) -> Dict[str, Any]:
    """
    Tool to record and save section details: section_ids, section_names of sections associated with a particular notebook Id.

    Args:
        notebook_id: The notebook ID of sections to store in section data session state
        section_ids: The Ids of the sections
        section_names: The corresponding section names of the sections
    """

    data = {
        "notebook_id": notebook_id,
        "sections":[
            {"section_id" : section_id, "section_name": section_name, }
            for section_id, section_name in zip(section_ids, section_names)
        ]
    }
       

    if "section:data" in tool_context.state and tool_context.state["section:data"]:
        tool_context.state["section:data"] += [data] 
    else:
        tool_context.state["section:data"] = [data]

    return {"status": "success"}


def retrieve_sectionsinfo(tool_context: ToolContext) -> Dict[str, Any]:
    """
    Tool to retrieve section data from session state.
    """

    section_data = tool_context.state.get("section:data", [])

    return {"status": "success", "section_data": section_data}


## Page operations


def save_pageinfo(
    tool_context: ToolContext, section_id: str, page_ids: List[str], page_names: List[str]
) -> Dict[str, Any]:
    """
    Tool to record and save page details: page_ids, page_names of pages associated with a particular section_id.

    Args:
        section_id: The section Id of the pages to store in page data session state
        page_ids: The Ids of the pages
        page_names: The corresponding page names of the pages
    """
   
    data = {
        "section_id": section_id,
        "pages":[
            {"page_id" : page_id, "page_name": page_name, }
            for page_id, page_name in zip(page_ids, page_names)
        ]
    }
       

    if "page:data" in tool_context.state and tool_context.state["page:data"]:
        tool_context.state["page:data"] += [data] 
    else:
        tool_context.state["page:data"] = [data]

    return {"status": "success"}


def retrieve_pageinfo(tool_context: ToolContext) -> Dict[str, Any]:
    """
    Tool to retrieve page data from session state.
    """

    page_data = tool_context.state.get("page:data", [])

    return {"status": "success", "page_data": page_data}


## Content operations

def save_content(
    tool_context: ToolContext, page_id: str, content: str
) -> Dict[str, Any]:
    """
    Tool to record and save content for a particular page_id.

    Args:
        page_id: The page_id of the contet to store in the content session state
        content: the content of the page_id
    """
   
    data = {
        "page_id": page_id,
        "content": content
    }
       

    if "content:data" in tool_context.state and tool_context.state["content:data"]:
        tool_context.state["content:data"] += [data] 
    else:
        tool_context.state["content:data"] = [data]

    return {"status": "success"}


def retrieve_content(tool_context: ToolContext) -> Dict[str, Any]:
    """
    Tool to retrieve content data from session state.
    """
    
    content_data = tool_context.state.get("content:data", [])
    
    return {"status": "success", "content_data": content_data}

### Creating 4 Agents that will be used as agent tools for the Root orchestrator agent
    - In earlier iterations, I tried same Agent for all the tasks: Notebook, sections, pages and content. But the Agent was struggling, not able to give deterministic answers and, sometimes gives completely wrong answers. So instead of one giant Agent, splitting it into 4 modular, more specialized agents.
    - Each Agent has exactly same task, just working on different domain and tools.   

In [775]:
notebook_agent = LlmAgent(
    model=Gemini(model=MODEL_NAME, retry_options=retry_config),
    name="noteBookAgent",
    description="""
    You are a context-aware orchestrator for OneNote Notebooks.
    
    YOUR STRICT EXECUTION PLAN:
    1. FIRST, call `retrieve_notebookinfo` to check if you already have the Notebook ID.
    2. IF (and only if) the ID is missing:
       a. Call `oneNoteAgent` to fetch the details".
       b. READ the output from `oneNoteAPIAgent`.
       c. EXTRACT the Notebook Name and Notebook ID from that output.
       d. IMMEDIATELY call `save_notebookinfo` with those details.
    3. FINALLY, answer the user's question using the information you found.
    
    If `oneNoteAPIAgent` tool gives output, DO NOT stop after step 2a, you MUST complete step 2d before finishing the turn.
    Otherwise skip steps 2c, 2d and return the message from the tool back to user.
    """,
    tools=[save_notebookinfo, retrieve_notebookinfo, AgentTool(oneNote_api_agent)],
)

section_agent = LlmAgent(
    model=Gemini(model=MODEL_NAME, retry_options=retry_config),
    name="sectionAgent",
    description="""
    You are a context-aware orchestrator for OneNote Sections.
    
    YOUR STRICT EXECUTION PLAN:
    1. FIRST, call `retrieve_sectionsinfo` to check if you already have associated Notebook ID for the Sections.
    2. IF (and only if) the Notebook ID for the section is missing:
       a. Call `oneNoteAgent` to fetch the details.
       b. READ the output from `oneNoteAPIAgent`.
       c. EXTRACT the Section Name, Section Id and Notebook ID from that output.
       d. IMMEDIATELY call `save_sectionsinfo` with those details.
    3. FINALLY, answer the user's question using the information you found.
    
    If `oneNoteAPIAgent` tool gives output, DO NOT stop after step 2a, you MUST complete step 2d before finishing the turn.
    Otherwise skip steps 2c, 2d and return the message from the tool back to user.
    """,
    tools=[save_sectionsinfo, retrieve_sectionsinfo, AgentTool(oneNote_api_agent)],
)

page_agent = LlmAgent(
    model=Gemini(model=MODEL_NAME, retry_options=retry_config),
    name="pageAgent",
    description="""
    You are a context-aware orchestrator for OneNote Pages.
    
    YOUR STRICT EXECUTION PLAN:
    1. FIRST, call `retrieve_pageinfo` to check if you already have associated Section ID for the Pages.
    2. IF (and only if) the Section ID for the page is missing:
       a. Call `oneNoteAgent` to fetch the details.
       b. READ the output from `oneNoteAPIAgent`.
       c. EXTRACT the Page Name, Page Id and Section ID from that output.
       d. IMMEDIATELY call `save_pageinfo` with those details.
    3. FINALLY, answer the user's question using the information you found.
    
    If `oneNoteAPIAgent` tool gives output, DO NOT stop after step 2a, you MUST complete step 2d before finishing the turn.
    Otherwise skip steps 2c, 2d and return the message from the tool back to user.
    """,
    tools=[save_pageinfo, retrieve_pageinfo, AgentTool(oneNote_api_agent)],
)

content_agent = LlmAgent(
    model=Gemini(model=MODEL_NAME, retry_options=retry_config),
    name="contentAgent",
    description="""
    You are a context-aware orchestrator for OneNote Contents.
    
    YOUR STRICT EXECUTION PLAN:
    1. FIRST, call `retrieve_content` to check if you already have associated Page ID for the content.
    2. IF (and only if) the Page ID for the content is missing:
       a. Call `oneNoteAgent` to fetch the details.
       b. READ the output from `oneNoteAPIAgent`.
       c. EXTRACT the content and Page Id from that output.
       d. IMMEDIATELY call `save_content` with those details.
    3. FINALLY, answer the user's question using the information you found.
    
    If `oneNoteAPIAgent` tool gives output, DO NOT stop after step 2a, you MUST complete step 2d before finishing the turn.
    Otherwise skip steps 2c, 2d and return the message from the tool back to user.
    """,
    tools=[save_content, retrieve_content, AgentTool(oneNote_api_agent)],
)

#### Creating the root orchestrator
    - This agent will see the user query and delegate the task to one agent tools defined above.
    - Running this agent in a session service to keep track of the previous data and save time on fetching data from API. 

In [776]:
APP_NAME = "default"
USER_ID = "default"
SESSION = "default"

oneNote_agent = LlmAgent(
    model=Gemini(model=MODEL_NAME, retry_options=retry_config),
    name="OneNoteAgent",
    description="""
    You are a context-aware orchestrator for OneNote Notebooks, Sections, Pages and Contents.
    
    Available tools:
    1. For user queries related to notebooks, you MUST use `noteBookAgent` tool.
    2. For user queries related to sections, you MUST use `sectionAgent` tool.
    3. For user quesries related to pages, you MUST use `page_agent` tool.
    4. For user quesries related to content, you MUST use `content_agent` tool.

    Return the response to users from these tools. No other response.
    """,
    tools=[AgentTool(section_agent), AgentTool(notebook_agent), AgentTool(page_agent), AgentTool(content_agent)],
)

session_service = InMemorySessionService()
onenote_runner = Runner(agent=oneNote_agent, app_name=APP_NAME, session_service=session_service)

### User Queries
    - The safegaurd I added to the API agent works and explains the only avilable data.

In [678]:
_ = await run_session(
    onenote_runner,
    [
        "Show me all the avilable notebooks in my OneNote account",
        "Show me details about notebook, Wonderland",
    ],
    "stateful-agentic-session",
)


 ### Session: stateful-agentic-session

User > Show me all the avilable notebooks in my OneNote account
Agent > OK. I have the notebook ID now. The notebook name is Google ADK and the notebook ID is 0-933A4B58C923BE21!s17b6741827f64337a35384cf5f6eb4ed.


User > Show me details about notebook, Wonderland
Agent > I am sorry, I am not able to get details about the notebook "Wonderland" as I only have information about "Google ADK".



### Session Management
    - The Agents are adding the data to the session and using when needed for later requests. 

In [679]:
_ = await showSessionState()

Session State Contents:
{
    "notebook:Id": "0-933A4B58C923BE21!s17b6741827f64337a35384cf5f6eb4ed",
    "notebook:Name": "Google ADK"
}


In [680]:
_ = await run_session(
    onenote_runner,
    [
       "Give me all the available sections in Google ADK notebook",
    ],
    "stateful-agentic-session",
)



 ### Session: stateful-agentic-session

User > Give me all the available sections in Google ADK notebook
Agent > The sections in the Google ADK notebook are: Chapters, Charactors, and Introduction.



In [511]:
_ = await showSessionState()

Session State Contents:
{
    "notebook:Id": "0-933A4B58C923BE21!s17b6741827f64337a35384cf5f6eb4ed",
    "notebook:Name": "Google ADK",
    "section:data": [
        {
            "notebook_id": "0-933A4B58C923BE21!s17b6741827f64337a35384cf5f6eb4ed",
            "sections": [
                {
                    "section_id": "0-933A4B58C923BE21!sf77f0c00e3a942e89ae040baed2d9b6d",
                    "section_name": "Chapters"
                },
                {
                    "section_id": "0-933A4B58C923BE21!sbab476bf76a94c629157118ebb829b06",
                    "section_name": "Charactors"
                },
                {
                    "section_id": "0-933A4B58C923BE21!s46f6175cc70f455dac1b984169129b9b",
                    "section_name": "Introduction"
                }
            ]
        }
    ]
}


In [686]:
_ = await run_session(
    onenote_runner,
    [
        "Show all the available pages for 'Chapters' section in the 'Google ADK' notebook",
        "Show all the available pages for 'Charactors' section in the 'Google ADK' notebook",
        "Show all the available pages for 'Introduction' section in the 'Google ADK' notebook",
    ],
    "stateful-agentic-session",
)


 ### Session: stateful-agentic-session

User > Show all the available pages for 'Chapters' section in the 'Google ADK' notebook
Agent > Here are the pages in the 'Chapters' section:
*   Year 6 — Enemy
*   Year 5 - Fragile Stability
*   Year 4 - Civil war
*   Year 3 - Inwards
*   Year 2 - Unify
*   Year 1 — The Hiding


User > Show all the available pages for 'Charactors' section in the 'Google ADK' notebook
Agent > Here are the pages for the 'Chapters' section: 
*   Arcs
*   Side
*   Humans
*   Main


User > Show all the available pages for 'Introduction' section in the 'Google ADK' notebook
Agent > Here are the pages for the 'Introduction' section: 
*   About
*   Setup
*   Lore




In [683]:
_ = await showSessionState()

Session State Contents:
{
    "notebook:Id": "0-933A4B58C923BE21!s17b6741827f64337a35384cf5f6eb4ed",
    "notebook:Name": "Google ADK",
    "section:data": [
        {
            "notebook_id": "0-933A4B58C923BE21!s17b6741827f64337a35384cf5f6eb4ed",
            "sections": [
                {
                    "section_id": "0-933A4B58C923BE21!sf77f0c00e3a942e89ae040baed2d9b6d",
                    "section_name": "Chapters"
                },
                {
                    "section_id": "0-933A4B58C923BE21!sbab476bf76a94c629157118ebb829b06",
                    "section_name": "Charactors"
                },
                {
                    "section_id": "0-933A4B58C923BE21!s46f6175cc70f455dac1b984169129b9b",
                    "section_name": "Introduction"
                }
            ]
        }
    ],
    "page:data": [
        {
            "section_id": "0-933A4B58C923BE21!s17b6741827f64337a35384cf5f6eb4ed",
            "pages": [
                {
             

In [ ]:
_ = await run_session(
    onenote_runner,
    [
        "Show the content of 'About' page for 'Chapters' section in the 'Google ADK' notebook",
    ],
    "stateful-agentic-session",
)


 ### Session: stateful-agentic-session

User > Show the content of 'About' page for 'Chapters' section in the 'Google ADK' notebook
Agent > Here is the content of 'About' page for 'Chapters' section in the 'Google ADK' notebook:
<html lang="en-US">
	<head>
		<title>About</title>
		<meta http-equiv="Content-Type" content="text/html; charset=utf-8" />
		<meta name="created" content="2025-12-01T13:12:00.0000000" />
	</head>
	<body data-absolute-enabled="true" style="font-family:Calibri;font-size:11pt">
		<div style="position:absolute;left:48px;top:115px;width:1184px">
			<p style="margin-top:0pt;margin-bottom:0pt">The Vherin are a once-thriving alien species who lived in peaceful harmony on their homeworld, Elyndor, until the catastrophic arrival of the Galthrax Dominion. The Galthrax—parasitic conquerors who feed on high-energy lifeforms—ravaged Elyndor, hunting the Vherin to the brink of extinction. In a desperate escape, only a few hundred Vherin fled across the galaxy. Their journey 

In [ ]:
_ = await showSessionState()

Session State Contents:
{
    "notebook:Id": "0-933A4B58C923BE21!s17b6741827f64337a35384cf5f6eb4ed",
    "notebook:Name": "Google ADK",
    "section:data": [
        {
            "notebook_id": "0-933A4B58C923BE21!s17b6741827f64337a35384cf5f6eb4ed",
            "sections": [
                {
                    "section_id": "0-933A4B58C923BE21!sf77f0c00e3a942e89ae040baed2d9b6d",
                    "section_name": "Chapters"
                },
                {
                    "section_id": "0-933A4B58C923BE21!sbab476bf76a94c629157118ebb829b06",
                    "section_name": "Charactors"
                },
                {
                    "section_id": "0-933A4B58C923BE21!s46f6175cc70f455dac1b984169129b9b",
                    "section_name": "Introduction"
                }
            ]
        }
    ],
    "page:data": [
        {
            "section_id": "0-933A4B58C923BE21!s17b6741827f64337a35384cf5f6eb4ed",
            "pages": [
                {
             

I was unable to save the other queries which had more complex questions after this. (The Kaggle notebook timed out unfortunately). However, the queries were trivial since it has all the data in the session to be used as context. 

#### I was throwing out some ideas on future improvements, couldn't complete it. They have the potential to be become even more powerful Agents.  

In [700]:
parallel_summerization_agent = ParallelAgent(
    name="parallelSummerizationAgent",
    sub_agents=[charactor_summerizer, chapters_summerizer, Introduction_summerizer],
)


root_agent = SequentialAgent(
    name="summerizationAgent",
    sub_agents=[parallel_summerization_agent, aggregator_agent],
)
